In [2]:
import matplotlib.pyplot as plt 
import pandas as pd
from pyarrow import parquet 
import numpy as np
import sys 
sys.path.append('/home/bekah/gPhoton2')
from gPhoton.pipeline import execute_pipeline
from astropy.io import fits
import os 

In [3]:
def get_selected_sources(eclipse, folder):
    """ get central sources from both fuv and nuv for the same eclipse. returns concat
    pd df with sources from both bands. """
    
    nuv_list_name = f"e{eclipse}-nd-full-0-photom-12_8.csv"
    fuv_list_name = f"e{eclipse}-fd-full-0-photom-12_8.csv"
    nuv_image_name = f"e{eclipse}-nd-full-0-rice.fits"
    fuv_image_name = f"e{eclipse}-fd-full-0-rice.fits"

    nuvexists, nuvpath = check_file_in_folder(folder, nuv_list_name) 
    fuvexists, fuvpath = check_file_in_folder(folder, fuv_list_name) 

    if nuvexists & fuvexists: 
        nuv = pd.read_csv(nuvpath)  
        nuv['band'] = 'nuv'
        fuv = pd.read_csv(fuvpath) 
        fuv['band'] = 'fuv'
        
        fuvandnuv = pd.concat([nuv, fuv])

        return fuvandnuv 

def check_file_in_folder(folder_path, file_name):
    try:
        file_path = os.path.join(folder_path, file_name)
        if os.path.exists(file_path):
            return True, file_path
        else:
            return False
    except FileNotFoundError:
        print(f"The folder '{folder_path}' does not exist.")
    except PermissionError:
        print(f"Permission error accessing folder '{folder_path}'.")
    except Exception as e:
        print(f"An error occurred: {e}")
        
def dbscan_group(fuvandnuv): 
    from sklearn.cluster import DBSCAN

    fuvandnuv_xy = list(zip(fuvandnuv['ra'],fuvandnuv['dec']))
    dbscan = DBSCAN(eps=.001, min_samples=2)
    labels = dbscan.fit_predict(fuvandnuv_xy)
    points_with_labels = zip(fuvandnuv_xy, labels)
    
    labelsdf = pd.DataFrame(points_with_labels, columns=['Point', 'Label'])
    labelsdf[['X', 'Y']] = labelsdf['Point'].apply(lambda x: pd.Series({'X': x[0], 'Y': x[1]}))
    labelsdf = labelsdf.drop('Point', axis=1)
    return labelsdf 


def calculate_vector(group):
    group['dx'] = group['ra'].diff()
    group['dy'] = group['dec'].diff()
    group['size_diff'] = group['equivalent_radius'].diff()
    return group 

def filter_middle_50_percent(group):
    lower_bound_r = group['ra'].quantile(0.25)
    upper_bound_r = group['ra'].quantile(0.75)
    lower_bound_d = group['dec'].quantile(0.25)
    upper_bound_d = group['dec'].quantile(0.75)
    filtered = group[(group['ra'] >= lower_bound_r) & (group['ra'] <= upper_bound_r)
                    & (group['dec'] >= lower_bound_d) & (group['dec'] <= upper_bound_d)]

    return filtered

def compare_bands(eclipse, eclipse_folder_path):

    e23330 = get_selected_sources(eclipse, eclipse_folder_path)

    color_mapping = {'nuv': 'red', 'fuv': 'blue'}

    e23330['Color'] = e23330['band'].map(color_mapping)

    # filter to middle of eclipse, change this 
    #selected_e23330 = e23330[(e23330['dec'] > -13.4) & (e23330['dec'] < -13.2) &
    #                           (e23330['ra'] > 354.2) & (e23330['ra'] < 354.4)]
    
    # Apply the function to each column
    
    #selected_e23330 = e23330.apply(filter_middle_30_percent)

    selected_e23330 = filter_middle_50_percent(e23330)

    label_e23330 = dbscan_group(selected_e23330)
    
    e23330_matches = label_e23330[(label_e23330['Label'] > 1)]
    
    label_e23330 = label_e23330.rename(columns={"X": "ra", "Y": "dec"})

    merged = pd.merge(selected_e23330, label_e23330, on=['ra', 'dec'], how='inner', validate="one_to_one")
    
    merged = merged[(merged['Label'] > 1)]
    
    label_groups = merged.sort_values(by=['Label'])
    
    return label_groups 

In [3]:
import cv2
import numpy as np

eclipse = '02506'

# Find corresponding points 
df = compare_bands(eclipse, f'/media/bekah/BekahA/eclipses_no_correction/e{eclipse}/')

In [4]:
fuv_df = df[df["band"] == "fuv"]
nuv_df = df[df["band"] == "nuv"]


def create_point_pairs_from_df(df):
    # Extract columns as NumPy arrays
    ra = df["ra"].values
    dec = df["dec"].values
    
    # converting to pixels? 
    ra = ra/1.5
    dec = dec/1.5
    
    # Concatenate columns into a single array
    points = np.column_stack((ra, dec))
    
    # Reshape array to Nx2
    point_pairs = points.reshape(-1, 2)
    
    return point_pairs

fuv_point_pairs = create_point_pairs_from_df(fuv_df)
nuv_point_pairs = create_point_pairs_from_df(nuv_df)



NameError: name 'df' is not defined

In [17]:

transformation_model = cv2.estimateAffinePartial2D(fuv_point_pairs, nuv_point_pairs)

In [18]:
transformation_model[0]

array([[ 1.00019144e+00, -1.80264626e-05, -3.98856733e-03],
       [ 1.80264626e-05,  1.00019144e+00, -2.12030998e-03]])

In [19]:
def ResizeWithAspectRatio(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, dim, interpolation=inter)

In [20]:
# Apply the transformation to one of the images
image = fits.open('/media/bekah/BekahA/eclipses_no_correction/e02506/e02506-fd-full-0-rice.fits')
image1 = image[1].data
aligned_image = cv2.warpAffine(image1, transformation_model[0], (image1.shape[1], image1.shape[0]))


In [21]:
hdu = fits.CompImageHDU(aligned_image, compression_type='RICE_1')

hdu.writeto('/media/bekah/BekahA/eclipses_no_correction/e02506/warped_e02506_scaled.fits', overwrite=True)

In [28]:
eclipses = ['02188','02313','02344','02372','02461'
            ,'02477','02492','02506','02564','02618']


def affine_eclipse(eclipse): 
    # Find corresponding points 
    df = compare_bands(eclipse, f'/media/bekah/BekahA/eclipses_no_correction/e{eclipse}/')
    fuv_df = df[df["band"] == "fuv"]
    nuv_df = df[df["band"] == "nuv"]

    fuv_point_pairs = create_point_pairs_from_df(fuv_df)
    nuv_point_pairs = create_point_pairs_from_df(nuv_df)

    transformation_model = cv2.estimateAffinePartial2D(fuv_point_pairs, nuv_point_pairs)

    # apply the transformation to one of the images
    image = fits.open(f'/media/bekah/BekahA/eclipses_no_correction/e{eclipse}/e{eclipse}-fd-full-0-rice.fits')
    image1 = image[1].data
    aligned_image = cv2.warpAffine(image1, transformation_model[0], (image1.shape[1], image1.shape[0]))

    # write image 
    hdu = fits.CompImageHDU(aligned_image, compression_type='RICE_1')
    hdu.writeto(f'/media/bekah/BekahA/eclipses_no_correction/e{eclipse}/warped_e{eclipse}_affine.fits', overwrite=True)

    return transformation_model[0]


for eclipse in eclipses: 
    try: 
        affine_eclipse(eclipse)
    except:
        print("something went wrong")
    
    

something went wrong
something went wrong
something went wrong
something went wrong


In [1]:
#eclipses = ['02188','02313','02344','02372','02461'
#            ,'02477','02492','02506','02564','02618']


eclipses = ['00922','01910','02117','02188','02344','02372',
            '02477','02492','02506','23054','23062','23068',
           '23085','23154','23258','23328','23330','23331',
           '23334','23335','23336','23340']

def scikit_affine_eclipse(eclipse): 
    # Find corresponding points 
    from skimage.transform import PiecewiseAffineTransform, warp

    df = compare_bands(eclipse, f'/media/bekah/BekahA/eclipses_no_correction/e{eclipse}/')
    fuv_df = df[df["band"] == "fuv"]
    nuv_df = df[df["band"] == "nuv"]

    fuv_point_pairs = create_point_pairs_from_df(fuv_df)
    nuv_point_pairs = create_point_pairs_from_df(nuv_df)

    print(len(nuv_point_pairs))
    tform = PiecewiseAffineTransform()
    tform.estimate(fuv_point_pairs, nuv_point_pairs)    
    
    # apply the transformation to one of the images
    image = fits.open(f'/media/bekah/BekahA/eclipses_no_correction/e{eclipse}/e{eclipse}-fd-full-0-rice.fits')
    image1 = image[1].data
    aligned_image = warp(image1, tform, output_shape=(image1.shape[1], image1.shape[0]))

    # write image 
    hdu = fits.CompImageHDU(aligned_image, compression_type='RICE_1')
    hdu.writeto(f'/media/bekah/BekahA/eclipses_no_correction/e{eclipse}/warped_e{eclipse}_scikit_affine.fits', overwrite=True)
    
    print(tform)
    return 


for eclipse in eclipses: 
    try: 
        scikit_affine_eclipse(eclipse)
    except:
        print("something went wrong")
    
    

something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong


In [29]:
eclipses = ['02188','02313','02344','02372','02461'
            ,'02477','02492','02506','02564','02618']


def homography_eclipse(eclipse): 
    # find points 
    df = compare_bands(eclipse, f'/media/bekah/BekahA/eclipses_no_correction/e{eclipse}/')
    fuv_df = df[df["band"] == "fuv"]
    nuv_df = df[df["band"] == "nuv"]

    fuv_point_pairs = create_point_pairs_from_df(fuv_df)
    nuv_point_pairs = create_point_pairs_from_df(nuv_df)

    transformation_model = cv2.estimateAffinePartial2D(fuv_point_pairs, nuv_point_pairs)
    homography, _ = cv2.findHomography(fuv_point_pairs, nuv_point_pairs)

    # apply transformation
    image = fits.open(f'/media/bekah/BekahA/eclipses_no_correction/e{eclipse}/e{eclipse}-fd-full-0-rice.fits')
    image1 = image[1].data
    height, width = image1.shape[:2]
    aligned_image = cv2.warpPerspective(image1, homography, (width, height))

    # write image 
    hdu = fits.CompImageHDU(aligned_image, compression_type='RICE_1')
    hdu.writeto(f'/media/bekah/BekahA/eclipses_no_correction/e{eclipse}/warped_e{eclipse}_homog.fits', overwrite=True)

    return transformation_model[0]


for eclipse in eclipses: 
    try: 
        homography_eclipse(eclipse)
    except:
        print("something went wrong")
    
    

something went wrong
something went wrong
something went wrong
something went wrong


In [25]:
    homography, _ = cv2.findHomography(fuv_point_pairs, nuv_point_pairs)


In [26]:
homography

array([[ 1.03981748e+00,  7.57341480e-02, -7.97326223e-01],
       [ 1.63853804e-03,  1.06920427e+00, -3.48736026e-01],
       [ 1.62359036e-04,  3.44256930e-03,  1.00000000e+00]])

In [ ]:
import numpy as np
import cv2

pixel_scale_ra = 1.5
pixel_scale_dec = 1.5 

# Sample celestial coordinates (RA, Dec) of reference point (e.g., center of the image)
ra_center = ...
dec_center = ...

# Convert celestial coordinates of reference point to pixel coordinates
pixel_x_center = (ra_center - ra_reference_point) / pixel_scale_ra
pixel_y_center = (dec_center - dec_reference_point) / pixel_scale_dec

# Apply the affine transformation to the reference point
transformed_pixel_x_center = affine_celestial[0, 0] * pixel_x_center + affine_celestial[0, 1] * pixel_y_center + affine_celestial[0, 2]
transformed_pixel_y_center = affine_celestial[1, 0] * pixel_x_center + affine_celestial[1, 1] * pixel_y_center + affine_celestial[1, 2]

# Calculate the translation in pixel coordinates
tx_pixel = transformed_pixel_x_center - pixel_x_center
ty_pixel = transformed_pixel_y_center - pixel_y_center

# Create the affine transformation matrix in pixel coordinates
affine_pixel = np.array([[affine_celestial[0, 0], affine_celestial[0, 1], tx_pixel],
                         [affine_celestial[1, 0], affine_celestial[1, 1], ty_pixel]])

# Apply the transformation to the image using OpenCV
# Read your image
image = cv2.imread('your_image.jpg')

# Apply the affine transformation
transformed_image = cv2.warpAffine(image, affine_pixel, (image.shape[1], image.shape[0]))

# Display the transformed image
cv2.imshow('Transformed Image', transformed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import data
from skimage.registration import phase_cross_correlation
from skimage.registration._phase_cross_correlation import _upsampled_dft
from scipy.ndimage import fourier_shift


image = data.camera()
shift = (-22.4, 13.32)
# The shift corresponds to the pixel offset relative to the reference image
offset_image = fourier_shift(np.fft.fftn(image), shift)
offset_image = np.fft.ifftn(offset_image)
print(f'Known offset (y, x): {shift}')

# pixel precision first
shift, error, diffphase = phase_cross_correlation(image, offset_image)

fig = plt.figure(figsize=(8, 3))
ax1 = plt.subplot(1, 3, 1)
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
ax3 = plt.subplot(1, 3, 3)

ax1.imshow(image, cmap='gray')
ax1.set_axis_off()
ax1.set_title('Reference image')

ax2.imshow(offset_image.real, cmap='gray')
ax2.set_axis_off()
ax2.set_title('Offset image')

# Show the output of a cross-correlation to show what the algorithm is
# doing behind the scenes
image_product = np.fft.fft2(image) * np.fft.fft2(offset_image).conj()
cc_image = np.fft.fftshift(np.fft.ifft2(image_product))
ax3.imshow(cc_image.real)
ax3.set_axis_off()
ax3.set_title("Cross-correlation")

plt.show()

print(f'Detected pixel offset (y, x): {shift}')

# subpixel precision
shift, error, diffphase = phase_cross_correlation(image, offset_image,
                                                  upsample_factor=100)

fig = plt.figure(figsize=(8, 3))
ax1 = plt.subplot(1, 3, 1)
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
ax3 = plt.subplot(1, 3, 3)

ax1.imshow(image, cmap='gray')
ax1.set_axis_off()
ax1.set_title('Reference image')

ax2.imshow(offset_image.real, cmap='gray')
ax2.set_axis_off()
ax2.set_title('Offset image')

# Calculate the upsampled DFT, again to show what the algorithm is doing
# behind the scenes.  Constants correspond to calculated values in routine.
# See source code for details.
cc_image = _upsampled_dft(image_product, 150, 100, (shift*100)+75).conj()
ax3.imshow(cc_image.real)
ax3.set_axis_off()
ax3.set_title("Supersampled XC sub-area")


plt.show()

print(f'Detected subpixel offset (y, x): {shift}')